In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score, explained_variance_score, median_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [ ]:
import requests
from io import StringIO

# Define the API endpoint URL
api_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

current_time = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S')

# Define the parameters
params = {
    'format': 'csv',
    'starttime': '1960-01-01',
    'endtime': current_time,
    'latitude': 43.75,
    'longitude': 77,
    'maxradiuskm': 500,
    'minmagnitude': 3,
    'orderby': 'time'
}

# Make the API request
response = requests.get(api_url, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Convert CSV data to Pandas DataFrame
    data = pd.read_csv(StringIO(response.text))
    
    # Save the DataFrame to a CSV file
    data.to_csv('earthquake_dataset.csv', index=False)

    # Print the first few rows of the DataFrame
    print(data)
else:
    # Print an error message if the request was not successful
    print(f"Error: {response.status_code} - {response.text}")


In [3]:
df = {'latitude': data.latitude, 'longitude': data.longitude, 'time': pd.to_datetime(data['time']),
     'depth': data.depth, 'year': pd.DatetimeIndex(data.time).year, 'month': pd.DatetimeIndex(data.time).month,'day': pd.DatetimeIndex(data.time).year,'hour': pd.DatetimeIndex(data.time).hour,'magnitude': data.mag}
df = pd.DataFrame(df)
df

,latitude,longitude,time,depth,year,month,day,hour,magnitude
0,41.0956,78.3679,2024-09-03 15:47:00.721000+00:00,13.353,2024,9,2024,15,4.30
1,41.3620,78.7243,2024-08-21 21:14:37.981000+00:00,10.000,2024,8,2024,21,4.50
2,39.9773,74.1667,2024-08-21 04:29:39.843000+00:00,10.000,2024,8,2024,4,4.30
3,42.2223,72.0394,2024-08-10 05:26:39.615000+00:00,17.642,2024,8,2024,5,4.20
4,40.0627,77.2831,2024-08-06 01:32:31.635000+00:00,10.000,2024,8,2024,1,3.90
...,...,...,...,...,...,...,...,...,...
2048,39.8780,77.4250,1961-12-30 07:08:37.030000+00:00,35.000,1961,12,1961,7,5.82
2049,39.7620,77.7120,1961-04-13 16:34:44.590000+00:00,35.000,1961,4,1961,16,7.04
2050,39.8130,77.7200,1961-04-06 01:33:51.550000+00:00,25.000,1961,4,1961,1,5.79
2051,39.8580,77.9100,1961-04-04 09:46:43.780000+00:00,20.000,1961,4,1961,9,6.34


In [ ]:
df['rolling_mean_magnitude'] = df['magnitude'].rolling(window=10, min_periods=1).mean()
df['time_since_last_hour'] = df['time'].diff().dt.total_seconds().div(3600).abs()
print(df.isna().sum())
df = df.fillna(0)
print(df.isna().sum())

from geopy.distance import geodesic

# Defining significant earthquakes
significant_threshold = 6.0
significant_earthquakes = df[df['magnitude'] >= significant_threshold].copy()

# Ensuring the time is in datetime format if not already
df['time'] = pd.to_datetime(df['time'])
significant_earthquakes['time'] = pd.to_datetime(significant_earthquakes['time'])
def find_closest_earthquake(row, significant_df):
    # Calculate distances
    distances = significant_df.apply(lambda x: geodesic((x['latitude'], x['longitude']), (row['latitude'], row['longitude'])).kilometers, axis=1)
    min_distance_index = distances.idxmin()
    closest_earthquake = significant_df.loc[min_distance_index]

    # Calculate time difference in days
    time_difference = abs((closest_earthquake['time'] - row['time']).total_seconds() / 86400)

    return pd.Series([closest_earthquake['time'], min_distance_index, distances[min_distance_index], time_difference])

# Apply the function to each row
df[['closest_eq_time', 'closest_eq_index', 'distance_to_closest_eq', 'time_diff_to_closest_eq']] = df.apply(find_closest_earthquake, significant_df=significant_earthquakes, axis=1)
print(df.isna().sum())


df = df.set_index('time', inplace=False)
df = df.drop(['hour', 'closest_eq_time', 'closest_eq_index'], axis=1)


In [7]:
# pip install geopy

In [5]:
X = df.drop('magnitude', axis=1)
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
#print(X)

y = df['magnitude']


In [ ]:
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")

# Show the plot
plt.show()

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# train_data = df[df['year'] < 2020]
# test_data = df[df['year'] >= 2020]

# X_train = train_data.drop('magnitude', axis=1)
# X_test = test_data.drop('magnitude', axis=1)

# # Scaling the features using RobustScaler
# scaler = RobustScaler()
# X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
# X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)

# y_train = train_data['magnitude']
# y_test = test_data['magnitude']

### KNN

In [ ]:
from skopt import BayesSearchCV
from sklearn.metrics import make_scorer
np.int = int
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Create KNeighborsRegressor model
knn_model = KNeighborsRegressor()

# Define parameter search space
param_space = {
    'n_neighbors': (5, 30),  # Adjust the upper limit accordingly
    'weights': ['uniform', 'distance'],
    'p': (1, 2),
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']
}
# Perform Bayesian optimization
bayes_search = BayesSearchCV(
    knn_model,
    param_space,
    cv=5,
    scoring=scorer, #'neg_mean_squared_error',
    n_iter=50,  # Number of parameter settings that are sampled
    random_state=42
)

# Fit the Bayesian search to the training data
bayes_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = bayes_search.best_params_

# Create a new KNeighborsRegressor model with the best hyperparameters
best_knn_model = KNeighborsRegressor(n_neighbors=best_params['n_neighbors'], weights=best_params['weights'])

# Fit the model to the training data
best_knn_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_knn = best_knn_model.predict(X_test)

# Calculate evaluation metrics
mae_knn = mean_absolute_error(y_test, y_pred_knn)
mse_knn = mean_squared_error(y_test, y_pred_knn)
mape_knn = mean_absolute_percentage_error(y_test, y_pred_knn)
r2_knn = r2_score(y_test, y_pred_knn)
evs_knn = explained_variance_score(y_test, y_pred_knn)
medae_knn = median_absolute_error(y_test, y_pred_knn)

# Print evaluation metrics and best hyperparameters
print("KNN Metrics:", best_params)
print("MAE:", mae_knn)
print("MSE:", mse_knn)
print("MAPE:", mape_knn)
print("R-squared:", r2_knn)
print("Explained Variance Score:", evs_knn)
print("Median Absolute Error:", medae_knn)


In [ ]:
import pandas as pd

# Convert the search results to a DataFrame
results_df = pd.DataFrame(bayes_search.cv_results_)

# Pivot the DataFrame to plot a heatmap for two hyperparameters (e.g., n_neighbors and p)
heatmap_data = results_df.pivot_table(values='mean_test_score', index='param_n_neighbors', columns='param_p')

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, cmap='viridis', annot=True)
plt.title('KNN Model Performance for n_neighbors and p')
plt.xlabel('p (Minkowski distance)')
plt.ylabel('n_neighbors')
plt.show()


## XGBoost Regression

In [ ]:
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from sklearn.metrics import make_scorer

# Create a pipeline with XGBoost Regressor
xgb_reg_model = XGBRegressor()

# Define the parameter search space for XGBoost
param_space_xgb = {
    'max_depth': (3, 10),  # Maximum tree depth
    'learning_rate': (0.01, 1.0, 'log-uniform'),  # Learning rate
    'n_estimators': (50, 200),  # Number of trees
    'gamma': (0.001, 1.0, 'log-uniform'),  # Regularization parameter
}

# Define the scoring metric (negative mean squared error)
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Perform Bayesian optimization for XGBoost
bayes_search_xgb = BayesSearchCV(
    xgb_reg_model,
    param_space_xgb,
    scoring=scorer,
    cv=5,  # 5-fold cross-validation
    n_iter=50,  # Number of parameter settings that are sampled
    random_state=42
)

# Fit the Bayesian search to the data
bayes_search_xgb.fit(X_train, y_train)

# Get the best parameters
best_params_xgb = bayes_search_xgb.best_params_

# Train the XGBoost Regressor with the best parameters
best_xgb_reg_model = XGBRegressor(**best_params_xgb)
best_xgb_reg_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = best_xgb_reg_model.predict(X_test)

# Calculate metrics
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mape_xgb = mean_absolute_percentage_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
evs_xgb = explained_variance_score(y_test, y_pred_xgb)
medae_xgb = median_absolute_error(y_test, y_pred_xgb)

# Print or use the metrics as needed
print("Best Parameters for XGBoost Regressor:", best_params_xgb)
print("XGBoost Regressor Metrics:")
print("MAE:", mae_xgb)
print("MSE:", mse_xgb)
print("MAPE:", mape_xgb)
print("R-squared:", r2_xgb)
print("Explained Variance Score:", evs_xgb)
print("Median Absolute Error:", medae_xgb)


### Desicion tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Define the parameter search space
param_space = {
    'max_depth': (1, 50),
    'min_samples_split': (2, 30),
    'min_samples_leaf': (1, 30),
    'max_features': ['sqrt', 'log2', None]
}
# Create the decision tree regressor
dt_model = DecisionTreeRegressor(random_state=42)

# Define the scoring metric (negative mean squared error)

# Perform Bayesian optimization
bayes_search = BayesSearchCV(
    dt_model,
    param_space,
    scoring=scorer,
    cv=5,  # 5-fold cross-validation
    n_iter=30,  # Number of parameter settings that are sampled
    random_state=42
)

# Fit the Bayesian search to the data
bayes_search.fit(X_train, y_train)

# Get the best parameters
best_params_dt = bayes_search.best_params_

# Train the Decision Tree with the best parameters
best_dt_model = DecisionTreeRegressor(
    max_depth=best_params_dt['max_depth'],
    min_samples_split=best_params_dt['min_samples_split'],
    min_samples_leaf=best_params_dt['min_samples_leaf'],
    random_state=42
)
best_dt_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_dt = best_dt_model.predict(X_test)

# Calculate metrics
mae_dt = mean_absolute_error(y_test, y_pred_dt)
mse_dt = mean_squared_error(y_test, y_pred_dt)
mape_dt = mean_absolute_percentage_error(y_test, y_pred_dt)
r2_dt= r2_score(y_test, y_pred_dt)
evs_dt = explained_variance_score(y_test, y_pred_dt)
medae_dt = median_absolute_error(y_test, y_pred_dt)

# Print or use the metrics as needed
print("Best Parameters for Decision Tree:", best_params_dt)
print("Decision Tree Metrics:")
print("MAE:", mae_dt)
print("MSE:", mse_dt)
print("MAPE:", mape_dt)
print("R-squared:", r2_dt)
print("Explained Variance Score:", evs_dt)
print("Median Absolute Error:", medae_dt)


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score, explained_variance_score, median_absolute_error
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV


# Define the parameter search space
param_space = {
    'n_estimators': (10, 200),
    'max_depth': (1, 30),
    'min_samples_split': (2, 100),
    'min_samples_leaf': (2, 100)
}

# Create the Random Forest regressor
rf_model = RandomForestRegressor(random_state=42)

# Define the scoring metric (negative mean squared error)


# Perform Bayesian optimization
bayes_search = BayesSearchCV(
    rf_model,
    param_space,
    scoring=scorer,
    cv=5,  # 5-fold cross-validation
    n_iter=50,  # Number of parameter settings that are sampled
    random_state=42
)

# Fit the Bayesian search to the data
bayes_search.fit(X_train, y_train)

# Get the best parameters
best_params_rf = bayes_search.best_params_

# Train the Random Forest with the best parameters
best_rf_model = RandomForestRegressor(**best_params_rf, random_state=42)
best_rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = best_rf_model.predict(X_test)

# Calculate metrics
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
evs_rf = explained_variance_score(y_test, y_pred_rf)
medae_rf = median_absolute_error(y_test, y_pred_rf)

# Print or use the metrics as needed
print("Best Parameters for Random Forest:", best_params_rf)
print("Random Forest Metrics:")
print("MAE:", mae_rf)
print("MSE:", mse_rf)
print("MAPE:", mape_rf)
print("R-squared:", r2_rf)
print("Explained Variance Score:", evs_rf)
print("Median Absolute Error:", medae_rf)


## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Define the parameter search space
param_space = {
    'n_estimators': (10, 200),
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'max_depth': (1, 30), 
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 10)
}

# Create the Gradient Boosting regressor
gb_model = GradientBoostingRegressor(random_state=42)

# Define the scoring metric (negative mean squared error)
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Perform Bayesian optimization
bayes_search = BayesSearchCV(
    gb_model,
    param_space,
    scoring=scorer,
    cv=5,  # 5-fold cross-validation
    n_iter=50,  # Number of parameter settings that are sampled
    random_state=42
)

# Fit the Bayesian search to the data
bayes_search.fit(X_train, y_train)

# Get the best parameters
best_params_gb = bayes_search.best_params_

# Train the Gradient Boosting with the best parameters
best_gb_model = GradientBoostingRegressor(**best_params_gb, random_state=42)
best_gb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_gb = best_gb_model.predict(X_test)

# Calculate metrics
mae_gb = mean_absolute_error(y_test, y_pred_gb)
mse_gb = mean_squared_error(y_test, y_pred_gb)
mape_gb = mean_absolute_percentage_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)
evs_gb = explained_variance_score(y_test, y_pred_gb)
medae_gb = median_absolute_error(y_test, y_pred_gb)

# Print or use the metrics as needed
print("Best Parameters for Gradient Boosting:", best_params_gb)
print("Gradient Boosting Metrics:")
print("MAE:", mae_gb)
print("MSE:", mse_gb)
print("MAPE:", mape_gb)
print("R-squared:", r2_gb)
print("Explained Variance Score:", evs_gb)
print("Median Absolute Error:", medae_gb)


## SVR

In [ ]:
from sklearn.svm import SVR

# Define the parameter search space
param_space = {
    'C': (1e-3, 1e+3, 'log-uniform'),
    'gamma': (1e-6, 1e+1, 'log-uniform'),
    # 'epsilon': (1e-6, 1e+1, 'log-uniform')
}

# Create the SVR model
svr_model = SVR()

# Define the scoring metric (negative mean squared error)
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Perform Bayesian optimization
bayes_search = BayesSearchCV(
    svr_model,
    param_space,
    scoring=scorer,
    cv=5,  # 5-fold cross-validation
    n_iter=50,  # Number of parameter settings that are sampled
    random_state=42
)

# Fit the Bayesian search to the data
bayes_search.fit(X_train, y_train)

# Get the best parameters
best_params_svr = bayes_search.best_params_

# Train the SVR with the best parameters
best_svr_model = SVR(**best_params_svr)
best_svr_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_svr = best_svr_model.predict(X_test)

# Calculate metrics
mae_svr = mean_absolute_error(y_test, y_pred_svr)
mse_svr = mean_squared_error(y_test, y_pred_svr)
mape_svr = mean_absolute_percentage_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)
evs_svr = explained_variance_score(y_test, y_pred_svr)
medae_svr = median_absolute_error(y_test, y_pred_svr)

# Print or use the metrics as needed
print("Best Parameters for SVR:", best_params_svr)
print("SVR Metrics:")
print("MAE:", mae_svr)
print("MSE:", mse_svr)
print("MAPE:", mape_svr)
print("R-squared:", r2_svr)
print("Explained Variance Score:", evs_svr)
print("Median Absolute Error:", medae_svr)


## Polynomial Regression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score, explained_variance_score, median_absolute_error
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import make_scorer

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter search space
param_space = {
    'poly__degree': (1, 5),
    'poly__interaction_only': [True, False]
}

# Create a pipeline with PolynomialFeatures and LinearRegression
poly_reg_model = Pipeline([
    ('poly', PolynomialFeatures()),
    ('reg', LinearRegression())
])

# Define the scoring metric (negative mean squared error)
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Perform Bayesian optimization
bayes_search = BayesSearchCV(
    poly_reg_model,
    param_space,
    scoring=scorer,
    cv=5,  # 5-fold cross-validation
    n_iter=50,  # Number of parameter settings that are sampled
    random_state=42
)

# Fit the Bayesian search to the data
bayes_search.fit(X_train, y_train)

# Get the best parameters
best_params_poly_reg = bayes_search.best_params_

# Train the Polynomial Regression with the best parameters
best_poly_reg_model = Pipeline([
    ('poly', PolynomialFeatures(degree=best_params_poly_reg['poly__degree'], interaction_only=best_params_poly_reg['poly__interaction_only'])),
    ('reg', LinearRegression())
])
best_poly_reg_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_poly_reg = best_poly_reg_model.predict(X_test)

# Calculate metrics
mae_poly_reg = mean_absolute_error(y_test, y_pred_poly_reg)
mse_poly_reg = mean_squared_error(y_test, y_pred_poly_reg)
mape_poly_reg = mean_absolute_percentage_error(y_test, y_pred_poly_reg)
r2_poly_reg = r2_score(y_test, y_pred_poly_reg)
evs_poly_reg = explained_variance_score(y_test, y_pred_poly_reg)
medae_poly_reg = median_absolute_error(y_test, y_pred_poly_reg)

# Print or use the metrics as needed
print("Best Parameters for Polynomial Regression:", best_params_poly_reg)
print("Polynomial Regression Metrics:")
print("MAE:", mae_poly_reg)
print("MSE:", mse_poly_reg)
print("MAPE:", mape_poly_reg)
print("R-squared:", r2_poly_reg)
print("Explained Variance Score:", evs_poly_reg)
print("Median Absolute Error:", medae_poly_reg)
